# 3DeeCellTracker Demo: single mode

This notebook shows how to use 3DeeCellTracker to track cells in single mode.

**The basic procedures:**
1. Initialize the parameters for tracking
2. Prepare images to be tracked and the pre-trained U-Net and FFN models.
3. Optimize segmentation parameters and segment the image at volume 1.
4. Correct the segmentation at volume 1 (in other image processing software)
5. Optimize tracking parameters.
6. Track following volumes.


**Please run folloing codes according to the instructions**
- Some codes can be skipped if unnecessary.

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
from matplotlib.animation import FuncAnimation, ArtistAnimation
from CellTracker.tracker import Tracker, save_tracker, load_tracker
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

## 1. Initialize the parameters for tracking
**Image parameters**
- volume: number of the volumes (time points) of the images to be tracked
- siz_xyz: size of each 3D image: (height, width, depth), unit: voxels
- z_xy_ratio: resolution (um/voxels) ratio between z (depth) and x-y plane
- z_scaling: (integer), the factor for interpolating the unbalanced z_xy resolution

**Segmentation parameters**
- noise_level: a value close to the averaged non-cell region intensity, which helps the program to ignore the background noises.
- min_size: the possible mimimal cell size, which helps the program to ignore small non-cell regions.
- Note: To change them after this initialization, please use ".set_segmentation()" which will delete the cached segmentation files

**Tracking parameters**
- beta_tk: set it higher/lower to get more coherent/independent predictions for cell positions.
- lambda_tk: set it higher/lower to get more coherent/independent predictions for cell positions.
- maxiter_tk: (integer) the number of iteration for the tracking algorithm (FFN+PR-GLS), the higher, the more accurate (but slower)

**Paths**
- folder_path: the path of the folder to store the data, model, and results.
    - "./xxx" indicates a folder with name "xxx" under the directory containing this jupyter notebook.
- image_name: the names of the images to be tracked. 
    - "aligned_t%03i_z%03i.tif" indicates file names like: "aligned_t002_z011.tif", "aligned_t502_z101.tif", etc.
- unet_model_file: the name of the pre-trained unet model file
- ffn_model_file: the name of the pre-trained ffn model file

In [ ]:
tracker = Tracker(
    volume_num=50, siz_xyz=(512, 1024, 21), z_xy_ratio=9.2, z_scaling=10, 
    noise_level=20, min_size=100, beta_tk=300, lambda_tk=0.1, maxiter_tk=20, 
    folder_path=os.path.abspath("./worm1"), image_name="aligned_t%03i_z%03i.tif", 
    unet_model_file="unet3_pretrained.h5", ffn_model_file="ffn_pretrained.h5")

**Create folders to store data, model, and results**
- /data: to store the images to be tracked
- /auto_vol1: to store the automatically generated segmentation results at volume 1
- /manual_vol1: to store the manually corrected segmentation results at volume 1
- /tracking_information: to store the coordinates information of the tracked cells
- /models: to store the pretrained and retrained models
- /unet: to store the cached predictions of cell regions
- /track_results_SingleMode: to store the tracking results (images of labels)
- /anim: to store the animation of the tracking process (used for diagnosis)
- /models/unet_weights: to store the retrained unet weights
- Note: in case one folder have been created, the program will not overwrite it.

In [ ]:
tracker.make_folders()

## 2. Prepare images and pre-trained models
**Prepare images**
- Move images to the created "/data" folder. The names of the images should be consistent with the parameter "image_name"
- Move the pre-trained 3D U-Net and FFN model files to the created "/model" folder

## 3. Optimize segmentation parameters and segment the image at volume 1.

**Modify the segmentation parameters (optional)**
- Please modify these segmentation parameters only when the following segmentation results are unsatisfied
- Note: the cached segmentation in "/unet" folder will be deleted after this modification

In [ ]:
tracker.set_segmentation(noise_level=20, min_size=100)

**Segment cells at volume 1**

In [ ]:
tracker.load_unet()
tracker.segment_vol1()

**Draw the results of segmentation (Max projection)**

In [ ]:
anim_seg = tracker.draw_segresult(percentile_high=99.8)

**Show segmentation in each layer**

In [ ]:
HTML(anim_seg)

## 4. Manually correct the segmentation at volume 1 and load it.
**Manual correction**
- Delete non-cell regions; Add cells not labeled; correct cell boundaries in other software (such as ITK-SNAP)
- Move the corrected segmentation (should be 2D images correponding to different z) to the created "/manual_vol1" folder

**Load the manually corrected segmentation**

In [ ]:
tracker.load_manual_seg()

**Re-train the U-Net using the manual segmentation (optional)**
- Please do this only if the prediction by U-Net is poor (e.g. many cells were not ignored or many non-cell regions were identified as cells).

In [ ]:
tracker.retrain_unet()

- Select an step with satisfied segmentation
- If the initial segmentation is better, set step=0

In [ ]:
tracker._select_weights(step=0)

**Interpolate cells to make more accurate/smooth cell boundary**

In [ ]:
tracker.interpolate_seg()
tracker.draw_manual_seg1()

**Initiate variables required for tracking**
- cal_subregions(): required by "accurate correction" in following procedures
- check_multicells(): if multiple separated cells were labeled as the same one, generate an error message.

In [ ]:
tracker.cal_subregions()
tracker.check_multicells()
tracker.load_ffn()
tracker.initiate_tracking()

## 5. Optimize tracking parameters. 
**Modify tracking parameters if the test result is not satisfied (optional)**

In [ ]:
tracker.set_tracking(beta_tk=300, lambda_tk=0.1, maxiter_tk=20)

**Test a matching between volume 1 and a target volume, and show the FFN + PR-GLS process by an animation (5 iterations)**
- Red circles: manually corrected cells in vol 1
- Blue cross: cells detected by 3D U-Net in target volume
- Blue arrows: predicted transformation (cell movements)

In [ ]:
anim_tracking, results = tracker.match(target_volume=50)
HTML(anim_tracking)

**Show the accurate correction after the FFN + PR-GLS transformation**
- Arrows show the correction of the cell positions
- Cells without arrow: no correction.

In [ ]:
tracker.draw_correction(*results[2:])

**Show the superimposed cells + labels before/after tracking**

In [ ]:
tracker.draw_overlapping(*results[:3])

## 6. Tracking following volumes
**Track and show the processes**

In [ ]:
%matplotlib notebook
from_volume=2
ax, fig = tracker.track_animation(from_volume)
tracker.track_and_confirm(from_volume, fig, ax)

**Show the processes as an animation (for diagnosis)**

In [ ]:
%matplotlib inline
track_anim = tracker.track_animation_replay(from_volume=2)
HTML(track_anim)